In [2]:
import pandas 
import torch
import numpy as np
import pandas as pd
from torch import nn
from sklearn.metrics import f1_score
from transformers import (
    XLMRobertaTokenizer,
    XLMRobertaModel,
    Trainer,
    TrainingArguments
)



/home/ka55gawy/semeval2025/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
label_df = pd.read_csv('../../task_data/subtask2_all_fine-grained_narratives.csv')

In [4]:
label_df

,narrative
0,CC: Amplifying Climate Fears: Amplifying exist...
1,CC: Amplifying Climate Fears: Doomsday scenari...
2,CC: Amplifying Climate Fears: Earth will be un...
3,CC: Amplifying Climate Fears: Whatever we do i...
4,CC: Amplifying Climate Fears: Other
...,...
91,URW: Speculating war outcomes: Russian army is...
92,URW: Speculating war outcomes: Russian army wi...
93,URW: Speculating war outcomes: Ukrainian army ...
94,URW: Speculating war outcomes: Other


In [5]:
train_df = pd.read_csv("legacy/train.csv")
train_df.head()

,filename,labels,narrative,subnarrative,subsubnarrative,text
0,EN_CC_200009.txt,CC: Green policies are geopolitical instrument...,CC,Green policies are geopolitical instruments,Green activities are a form of neo-colonialism,Climate Hypocrisy Exposed: EV Battery Producti...
1,EN_CC_200262.txt,CC: Criticism of institutions and authorities:...,CC,Criticism of institutions and authorities,Criticism of political organizations and figures,DeSantis brags his climate change plan is scra...
2,EN_CC_200012.txt,CC: Criticism of institutions and authorities:...,CC,Criticism of institutions and authorities,Criticism of national governments,West silences activists at home while promotin...
3,EN_UA_021123.txt,URW: Discrediting Ukraine: Situation in Ukrain...,URW,Discrediting Ukraine,Situation in Ukraine is hopeless,CIA's Burns & SecDef Austin In Separate 'Urgen...
4,EN_CC_200048.txt,CC: Hidden plots by secret schemes of powerful...,CC,Hidden plots by secret schemes of powerful groups,Other,New York AG Letitia James wages WAR on beef in...


In [6]:
val_df = pd.read_csv("legacy/val.csv")
val_df.head()

,filename,labels,narrative,subnarrative,subsubnarrative,text
0,EN_UA_300009.txt,Other,Other,Other,Other,Russia: Clashes erupt in Bashkortostan as righ...
1,EN_UA_019640.txt,URW: Russia is the Victim: The West is russoph...,URW,Russia is the Victim,The West is russophobic,"After North Korea’s Kim Jong Un, Putin and Xi ..."
2,EN_CC_200166.txt,CC: Criticism of climate movement: Ad hominem ...,CC,Criticism of climate movement,Ad hominem attacks on key activists,Greta Thunberg is arrested at London protest. ...
3,EN_UA_300153.txt,URW: Discrediting Ukraine: Discrediting Ukrain...,URW,Discrediting Ukraine,Discrediting Ukrainian government and official...,Exposed: CIA Plotting WW3 With Russia! False F...
4,EN_UA_300153.txt,URW: Blaming the war on others rather than the...,URW,Blaming the war on others rather than the invader,The West are the aggressors,Exposed: CIA Plotting WW3 With Russia! False F...


In [7]:
train_df["narrative"] = train_df["narrative"] + ":" + train_df["subnarrative"]
train_df["subnarrative"] = train_df["subsubnarrative"]

In [8]:
val_df["narrative"] = val_df["narrative"] + ":" + val_df["subnarrative"]
val_df["subnarrative"] = val_df["subsubnarrative"]

In [9]:
val_df.head()

,filename,labels,narrative,subnarrative,subsubnarrative,text
0,EN_UA_300009.txt,Other,Other:Other,Other,Other,Russia: Clashes erupt in Bashkortostan as righ...
1,EN_UA_019640.txt,URW: Russia is the Victim: The West is russoph...,URW:Russia is the Victim,The West is russophobic,The West is russophobic,"After North Korea’s Kim Jong Un, Putin and Xi ..."
2,EN_CC_200166.txt,CC: Criticism of climate movement: Ad hominem ...,CC:Criticism of climate movement,Ad hominem attacks on key activists,Ad hominem attacks on key activists,Greta Thunberg is arrested at London protest. ...
3,EN_UA_300153.txt,URW: Discrediting Ukraine: Discrediting Ukrain...,URW:Discrediting Ukraine,Discrediting Ukrainian government and official...,Discrediting Ukrainian government and official...,Exposed: CIA Plotting WW3 With Russia! False F...
4,EN_UA_300153.txt,URW: Blaming the war on others rather than the...,URW:Blaming the war on others rather than the ...,The West are the aggressors,The West are the aggressors,Exposed: CIA Plotting WW3 With Russia! False F...


In [10]:
val_df[["text","narrative","subnarrative"]].isna().sum()

text            0
narrative       0
subnarrative    0
dtype: int64

In [11]:
for i in val_df.iterrows():
    print(i)

(0, filename                                            EN_UA_300009.txt
labels                                                         Other
narrative                                                Other:Other
subnarrative                                                   Other
subsubnarrative                                                Other
text               Russia: Clashes erupt in Bashkortostan as righ...
Name: 0, dtype: object)
(1, filename                                            EN_UA_019640.txt
labels             URW: Russia is the Victim: The West is russoph...
narrative                                   URW:Russia is the Victim
subnarrative                                 The West is russophobic
subsubnarrative                              The West is russophobic
text               After North Korea’s Kim Jong Un, Putin and Xi ...
Name: 1, dtype: object)
(2, filename                                            EN_CC_200166.txt
labels             CC: Criticism of climate

In [24]:
available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
available_gpus

[<torch.cuda.device at 0x7f5ecf997bc0>, <torch.cuda.device at 0x7f5ecefdeff0>]

In [23]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, hamming_loss, precision_score, recall_score
import torch.nn.functional as F
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Constants
MAX_LENGTH = 512
BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 2e-5
ACCUMULATION_STEPS = 4

# Dataset and Model classes remain the same as before...
# [Previous HierarchicalDataset and HierarchicalClassifier classes]
# Constants
MAX_LENGTH = 512
BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 2e-5
ACCUMULATION_STEPS = 4

class HierarchicalDataset(Dataset):
    def __init__(self, texts, narrative_labels, subnarrative_labels, tokenizer):
        self.texts = texts
        self.narrative_labels = narrative_labels
        self.subnarrative_labels = subnarrative_labels
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        # Tokenize the text
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=MAX_LENGTH,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'narrative_labels': torch.FloatTensor(self.narrative_labels[idx]),
            'subnarrative_labels': torch.FloatTensor(self.subnarrative_labels[idx])
        }

class HierarchicalClassifier(nn.Module):
    def __init__(self, num_narratives, num_subnarratives):
        super().__init__()
        self.xlm_roberta = XLMRobertaModel.from_pretrained('xlm-roberta-base')
        
        self.dropout = nn.Dropout(0.1)
        
        # Shared layers
        self.shared = nn.Sequential(
            nn.Linear(768, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 256)
        )
        
        # Classification heads
        self.narrative_classifier = nn.Linear(256, num_narratives)
        self.subnarrative_classifier = nn.Linear(256 + num_narratives, num_subnarratives)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.xlm_roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]  # CLS token
        
        # Shared features
        shared_features = self.shared(pooled_output)
        
        # Narrative prediction
        narrative_logits = self.narrative_classifier(shared_features)
        narrative_probs = torch.sigmoid(narrative_logits)
        
        # Concatenate features with narrative probabilities for subnarrative prediction
        subnarrative_input = torch.cat([shared_features, narrative_probs], dim=1)
        subnarrative_logits = self.subnarrative_classifier(subnarrative_input)
        
        return narrative_logits, subnarrative_logits

def compute_metrics(y_true, y_pred):
    """Compute various metrics for multilabel classification"""
    return {
        'micro_f1': f1_score(y_true, y_pred, average='micro'),
        'macro_f1': f1_score(y_true, y_pred, average='macro'),
        'weighted_f1': f1_score(y_true, y_pred, average='weighted'),
        'hamming_loss': hamming_loss(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='micro'),
        'recall': recall_score(y_true, y_pred, average='micro')
    }

def train_model(train_df, val_df):
    # Load data
    # train_df = pd.read_csv('legacy/train.csv')
    # val_df = pd.read_csv('legacy/val.csv')
    
    # Process multiple labels
    # for df in [train_df, val_df]:
    #     df['narrative'] = df['narrative'].str.split('|')
    #     df['subnarrative'] = df['subnarrative'].str.split('|')
    
    # Create label encoders
    narrative_mlb = MultiLabelBinarizer()
    subnarrative_mlb = MultiLabelBinarizer()
    
    # Fit on training data
    train_narrative_labels = narrative_mlb.fit_transform(train_df['narrative'])
    train_subnarrative_labels = subnarrative_mlb.fit_transform(train_df['subnarrative'])
    
    # Transform validation data
    val_narrative_labels = narrative_mlb.transform(val_df['narrative'])
    val_subnarrative_labels = subnarrative_mlb.transform(val_df['subnarrative'])
    
    # Initialize tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
    
    # Create datasets
    train_dataset = HierarchicalDataset(
        train_df['text'], 
        train_narrative_labels, 
        train_subnarrative_labels, 
        tokenizer
    )
    
    val_dataset = HierarchicalDataset(
        val_df['text'], 
        val_narrative_labels, 
        val_subnarrative_labels, 
        tokenizer
    )
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    # Initialize model
    model = HierarchicalClassifier(
        num_narratives=len(narrative_mlb.classes_),
        num_subnarratives=len(subnarrative_mlb.classes_)
    )
    model.to(device)
    
    # Initialize optimizer and scheduler
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2)
    
    # Training loop
    best_val_loss = float('inf')
    for epoch in range(EPOCHS):
        print(f'\nEpoch {epoch + 1}/{EPOCHS}')
        print('-' * 60)
        
        # Training phase
        model.train()
        total_loss = 0
        optimizer.zero_grad()
        
        train_narrative_preds = []
        train_narrative_trues = []
        train_subnarrative_preds = []
        train_subnarrative_trues = []
        
        train_bar = tqdm(train_loader, desc='Training')
        for batch_idx, batch in enumerate(train_bar):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            narrative_labels = batch['narrative_labels'].to(device)
            subnarrative_labels = batch['subnarrative_labels'].to(device)
            
            # Forward pass
            narrative_logits, subnarrative_logits = model(input_ids, attention_mask)
            
            # Calculate loss
            narrative_loss = F.binary_cross_entropy_with_logits(narrative_logits, narrative_labels)
            subnarrative_loss = F.binary_cross_entropy_with_logits(subnarrative_logits, subnarrative_labels)
            
            loss = narrative_loss + subnarrative_loss
            
            # Backward pass
            loss = loss / ACCUMULATION_STEPS
            loss.backward()
            
            if (batch_idx + 1) % ACCUMULATION_STEPS == 0:
                optimizer.step()
                optimizer.zero_grad()
            
            total_loss += loss.item() * ACCUMULATION_STEPS
            
            # Store predictions
            narrative_preds = (torch.sigmoid(narrative_logits) > 0.5).cpu().numpy()
            subnarrative_preds = (torch.sigmoid(subnarrative_logits) > 0.5).cpu().numpy()
            
            train_narrative_preds.extend(narrative_preds)
            train_narrative_trues.extend(narrative_labels.cpu().numpy())
            train_subnarrative_preds.extend(subnarrative_preds)
            train_subnarrative_trues.extend(subnarrative_labels.cpu().numpy())
            
            # Update progress bar
            train_bar.set_postfix({'loss': loss.item() * ACCUMULATION_STEPS})
        
        avg_train_loss = total_loss / len(train_loader)
        
        # Compute training metrics
        train_narrative_metrics = compute_metrics(
            np.array(train_narrative_trues), 
            np.array(train_narrative_preds)
        )
        train_subnarrative_metrics = compute_metrics(
            np.array(train_subnarrative_trues), 
            np.array(train_subnarrative_preds)
        )
        
        # Validation phase
        model.eval()
        val_loss = 0
        
        val_narrative_preds = []
        val_narrative_trues = []
        val_subnarrative_preds = []
        val_subnarrative_trues = []
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc='Validation')
            for batch in val_bar:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                narrative_labels = batch['narrative_labels'].to(device)
                subnarrative_labels = batch['subnarrative_labels'].to(device)
                
                narrative_logits, subnarrative_logits = model(input_ids, attention_mask)
                
                narrative_loss = F.binary_cross_entropy_with_logits(narrative_logits, narrative_labels)
                subnarrative_loss = F.binary_cross_entropy_with_logits(subnarrative_logits, subnarrative_labels)
                
                batch_loss = narrative_loss + subnarrative_loss
                val_loss += batch_loss.item()
                
                # Store predictions
                narrative_preds = (torch.sigmoid(narrative_logits) > 0.5).cpu().numpy()
                subnarrative_preds = (torch.sigmoid(subnarrative_logits) > 0.5).cpu().numpy()
                
                val_narrative_preds.extend(narrative_preds)
                val_narrative_trues.extend(narrative_labels.cpu().numpy())
                val_subnarrative_preds.extend(subnarrative_preds)
                val_subnarrative_trues.extend(subnarrative_labels.cpu().numpy())
                
                val_bar.set_postfix({'loss': batch_loss.item()})
        
        avg_val_loss = val_loss / len(val_loader)
        
        # Compute validation metrics
        val_narrative_metrics = compute_metrics(
            np.array(val_narrative_trues), 
            np.array(val_narrative_preds)
        )
        val_subnarrative_metrics = compute_metrics(
            np.array(val_subnarrative_trues), 
            np.array(val_subnarrative_preds)
        )
        
        # Print metrics
        print('\nTraining Metrics:')
        print(f'Loss: {avg_train_loss:.4f}')
        print('\nNarrative Level:')
        for metric, value in train_narrative_metrics.items():
            print(f'{metric}: {value:.4f}')
        print('\nSubnarrative Level:')
        for metric, value in train_subnarrative_metrics.items():
            print(f'{metric}: {value:.4f}')
        
        print('\nValidation Metrics:')
        print(f'Loss: {avg_val_loss:.4f}')
        print('\nNarrative Level:')
        for metric, value in val_narrative_metrics.items():
            print(f'{metric}: {value:.4f}')
        print('\nSubnarrative Level:')
        for metric, value in val_subnarrative_metrics.items():
            print(f'{metric}: {value:.4f}')
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'narrative_mlb': narrative_mlb,
                'subnarrative_mlb': subnarrative_mlb,
                'best_val_loss': best_val_loss
            }, 'models/xlmroberta/best_model.pt')
            print('\nBest model saved!')
        
        print('-' * 60)

train_model(train_df, val_df)


Epoch 1/10
------------------------------------------------------------


Training:   1%|          | 1/93 [00:00<00:30,  3.03it/s, loss=1.38]

KeyboardInterrupt



In [26]:
model.predict("I am a good")

NameError: name 'model' is not defined

In [25]:
def load_model_for_prediction(model_path):
    """Load the trained model and label encoders"""
    # Load checkpoint
    checkpoint = torch.load(model_path)
    
    # Initialize model
    model = HierarchicalClassifier(
        num_narratives=len(checkpoint['narrative_mlb'].classes_),
        num_subnarratives=len(checkpoint['subnarrative_mlb'].classes_)
    )
    
    # Load state dict
    model.load_state_dict(checkpoint['model_state_dict'])
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    return model, checkpoint['narrative_mlb'], checkpoint['subnarrative_mlb']

def predict_labels(texts, model_path='best_model.pt', threshold=0.5, batch_size=8):
    """
    Predict narratives and subnarratives for given texts
    
    Args:
        texts: List of strings or single string
        model_path: Path to saved model checkpoint
        threshold: Confidence threshold for predictions
        batch_size: Batch size for processing
    
    Returns:
        List of dictionaries containing predictions and confidence scores
    """
    # Handle single text input
    if isinstance(texts, str):
        texts = [texts]
    
    # Load model and label encoders
    model, narrative_mlb, subnarrative_mlb = load_model_for_prediction(model_path)
    
    # Initialize tokenizer
    tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
    
    # Create dataset
    class PredictionDataset(Dataset):
        def __init__(self, texts, tokenizer):
            self.texts = texts
            self.tokenizer = tokenizer
        
        def __len__(self):
            return len(self.texts)
        
        def __getitem__(self, idx):
            encoding = self.tokenizer.encode_plus(
                str(self.texts[idx]),
                add_special_tokens=True,
                max_length=MAX_LENGTH,
                return_token_type_ids=False,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten()
            }
    
    # Create dataloader
    dataset = PredictionDataset(texts, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size)
    
    # Lists to store predictions
    all_narrative_preds = []
    all_narrative_probs = []
    all_subnarrative_preds = []
    all_subnarrative_probs = []
    
    # Make predictions
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Making predictions"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            
            # Get model predictions
            narrative_logits, subnarrative_logits = model(input_ids, attention_mask)
            
            # Get probabilities
            narrative_probs = torch.sigmoid(narrative_logits)
            subnarrative_probs = torch.sigmoid(subnarrative_logits)
            
            # Get binary predictions
            narrative_preds = (narrative_probs > threshold).cpu().numpy()
            subnarrative_preds = (subnarrative_probs > threshold).cpu().numpy()
            
            # Store predictions and probabilities
            all_narrative_preds.extend(narrative_preds)
            all_narrative_probs.extend(narrative_probs.cpu().numpy())
            all_subnarrative_preds.extend(subnarrative_preds)
            all_subnarrative_probs.extend(subnarrative_probs.cpu().numpy())
    
    # Process predictions
        results = []
    for idx, (text, nar_preds, nar_probs, sub_preds, sub_probs) in enumerate(zip(
        texts, all_narrative_preds, all_narrative_probs, 
        all_subnarrative_preds, all_subnarrative_probs
    )):
        # Find main narrative predictions
        narrative_predictions = [
            (label.split(': ', 1)[0], prob) 
            for label, pred, prob in zip(narrative_mlb.classes_, nar_preds, nar_probs) 
            if pred
        ]
        
        # Find subnarrative predictions with their parent narratives
        subnarrative_predictions = [
            (label.rsplit(': ', 1)[-1], prob)  # Get only the last part after the last ':'
            for label, pred, prob in zip(subnarrative_mlb.classes_, sub_preds, sub_probs) 
            if pred
        ]
        
        narrative_predictions.sort(key=lambda x: x[1], reverse=True)
        subnarrative_predictions.sort(key=lambda x: x[1], reverse=True)
        
        results.append({
            'text': text,
            'narratives': [
                {
                    'label': label,
                    'confidence': float(prob)
                }
                for label, prob in narrative_predictions
            ],
            'subnarratives': [
                {
                    'label': label,
                    'confidence': float(prob)
                }
                for label, prob in subnarrative_predictions
            ]
        })
    
    return results

# Example usage:
def predict_example():
    # Example texts
    texts = [
        "Climate change is causing irreversible damage to our planet.",
        "The war in Ukraine has led to significant geopolitical tensions."
    ]
    
    # Make predictions
    predictions = predict_labels(texts, model_path="models/xlmroberta/best_model.pt")
    
    # Print results
    for pred in predictions:
        print("\nText:", pred['text'])
        print("\nNarratives:")
        for nar in pred['narratives']:
            print(f"- {nar['label']} (confidence: {nar['confidence']:.3f})")
        print("\nSubnarratives:")
        for sub in pred['subnarratives']:
            print(f"- {sub['label']} (confidence: {sub['confidence']:.3f})")
        print("-" * 80)

if __name__ == "__main__":
    predict_example()

OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB. GPU 0 has a total capacity of 23.61 GiB of which 63.06 MiB is free. Process 976664 has 7.72 GiB memory in use. Process 1004665 has 6.89 GiB memory in use. Including non-PyTorch memory, this process has 7.81 GiB memory in use. Of the allocated memory 7.21 GiB is allocated by PyTorch, and 152.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [26]:
train_df

,filename,labels,narrative,subnarrative,subsubnarrative,text
0,EN_CC_200009.txt,CC: Green policies are geopolitical instrument...,CC:Green policies are geopolitical instruments,Green activities are a form of neo-colonialism,Green activities are a form of neo-colonialism,Climate Hypocrisy Exposed: EV Battery Producti...
1,EN_CC_200262.txt,CC: Criticism of institutions and authorities:...,CC:Criticism of institutions and authorities,Criticism of political organizations and figures,Criticism of political organizations and figures,DeSantis brags his climate change plan is scra...
2,EN_CC_200012.txt,CC: Criticism of institutions and authorities:...,CC:Criticism of institutions and authorities,Criticism of national governments,Criticism of national governments,West silences activists at home while promotin...
3,EN_UA_021123.txt,URW: Discrediting Ukraine: Situation in Ukrain...,URW:Discrediting Ukraine,Situation in Ukraine is hopeless,Situation in Ukraine is hopeless,CIA's Burns & SecDef Austin In Separate 'Urgen...
4,EN_CC_200048.txt,CC: Hidden plots by secret schemes of powerful...,CC:Hidden plots by secret schemes of powerful ...,Other,Other,New York AG Letitia James wages WAR on beef in...
...,...,...,...,...,...,...
739,EN_CC_200245.txt,CC: Controversy about green technologies: Rene...,CC:Controversy about green technologies,Renewable energy is unreliable,Renewable energy is unreliable,Sen. Tommy Tuberville: “Climate Change is the ...
740,EN_CC_200289.txt,CC: Downplaying climate change: Temperature in...,CC:Downplaying climate change,Temperature increase does not have significant...,Temperature increase does not have significant...,Ocean acidification claims of climate cult are...
741,EN_UA_300036.txt,Other,Other:Other,Other,Other,US to Help Ukraine Print School Textbooks as R...
742,EN_UA_026697.txt,URW: Distrust towards Media: Western media is ...,URW:Distrust towards Media,Western media is an instrument of propaganda,Western media is an instrument of propaganda,The Eurocrats Fear That Fico's Attempted Assas...


In [13]:

from sklearn.preprocessing import MultiLabelBinarizer

In [14]:
narrative_mlb = MultiLabelBinarizer()
subnarrative_mlb = MultiLabelBinarizer()

# Fit on training data
train_narrative_labels = narrative_mlb.fit_transform(train_df['narrative'])
train_subnarrative_labels = subnarrative_mlb.fit_transform(train_df['subnarrative'])

# Transform validation data
val_narrative_labels = narrative_mlb.transform(val_df['narrative'])
val_subnarrative_labels = subnarrative_mlb.transform(val_df['subnarrative'])



In [17]:
train_narrative_labels

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 0, 0, 1]], shape=(744, 44))